In [ ]:
from dataset import EarthQuakeDataModule
from model import EarthQuakeModel
from pytorch_lightning import Trainer
from joblib import load
from sklearn.metrics import mean_absolute_error, accuracy_score

In [3]:
dm = EarthQuakeDataModule(file="data/earthquakes.h5", split_file="data/earthquakes_split.parquet", magnitudes_file="data/earthquakes.parquet", num_workers=8, batch_size=16, fill_nan=0, post_only=True)

# Deep Learning

In [3]:
CHECKPOINT = ""
model = EarthQuakeModel.load_from_checkpoint(CHECKPOINT, strict=False)

In [ ]:
trainer = Trainer(devices=[1])
out = trainer.test(model, datamodule=dm)

# Classical

In [4]:
CHECKPOINT = ""
model = load(CHECKPOINT)

In [ ]:
dm.setup("test")
test_sample = [dm.test_dataset[i]["sample"].flatten() for i in range(10)]
model[0].transform(test_sample).shape, test_sample[0].shape

In [ ]:
KEY = "label"
dm.setup("test")
prediction_gt = []
model.verbose = False
for sample in dm.test_dataset:
    gt = sample[KEY]
    pred = model.predict(sample["sample"].flatten().reshape(1, -1))
    prediction_gt.append((pred, gt))

In [ ]:
mae = mean_absolute_error([x[1] for x in prediction_gt], [x[0] for x in prediction_gt])
acc = accuracy_score([x[1] > 1 for x in prediction_gt], [x[0] > 0 for x in prediction_gt])

mae, acc

In [ ]:
accuracy_score([x[1] for x in prediction_gt], [x[0] for x in prediction_gt])